This notebook extracts training features from the Open Data Cube (ODC) of Sentinel-2 multispectral images, using unfiltered) training data in a previous year. The features include bi-monthly geomedian of Sentinel-2 bands.

### load packages and get number of cpus

In [ ]:
%matplotlib inline

import datacube
import warnings
import numpy as np
import geopandas as gpd
import pandas as pd
import xarray as xr
from odc.io.cgroups import get_cpu_quota
from odc.algo import xr_geomedian
from deafrica_tools.plotting import map_shapefile
from deafrica_tools.datahandling import load_ard
from deafrica_tools.bandindices import calculate_indices
from deafrica_tools.classification import collect_training_data

ncpus=round(get_cpu_quota())
print('ncpus = '+str(ncpus))

### input files and parameters

In [ ]:
training_data_path = 'Data/train_poly_848_20171124.shp'
class_attr = 'Class_I' # class label in integer format
output_crs='epsg:32736' # WGS84/UTM Zone 36S

### load input data

In [ ]:
# Load input data shapefile
training_data= gpd.read_file(training_data_path) # read training points as geopandas dataframe
training_data=training_data[[class_attr,'geometry']] # select attributes
# Plot first five rows
training_data.head()

### class merging

In [ ]:
dict_map={'Tree crops': 11, 'Field crops': 12, 'Forest plantations': 21, 'Grassland': 31, 'Shrubland': 33, 'Aquatic or regularly flooded shrublands': 41,
 'Aquatic or regularly flooded herbaceous vegetation': 42, 'Water body': 44, 'Settlements': 51, 'Bare soils': 61,
 'Bare rocks': 62, 'Mangrove': 70, 'Mecrusse': 71, 'Closed broadleaved (Semi-) evergreen mountaineous forest': 72,
 'Gallery forest': 73, 'Broadleaved (Semi-) deciduous closed forest': 74, 'Mopane': 75, 'Open broadleaved (Semi-) evergreen mountaineous forest': 76,
 'Coastal open woody vegetation': 77, 'Mopane open': 78, 'Miombo open': 79}

training_data.loc[training_data[class_attr]==dict_map['Shrubland'],class_attr]=dict_map['Grassland']
training_data.loc[training_data[class_attr]==dict_map['Aquatic or regularly flooded herbaceous vegetation'],
                  class_attr]=dict_map['Aquatic or regularly flooded shrublands']
training_data.loc[training_data[class_attr]==dict_map['Bare rocks'],class_attr]=dict_map['Bare soils']
training_data.loc[(training_data[class_attr]==dict_map['Gallery forest'])|
                  (training_data[class_attr]==dict_map['Open broadleaved (Semi-) evergreen mountaineous forest'])
                  |(training_data[class_attr]==dict_map['Coastal open woody vegetation']),class_attr]=dict_map['Closed broadleaved (Semi-) evergreen mountaineous forest']
training_data.loc[training_data[class_attr]==dict_map['Mopane open'],class_attr]=dict_map['Mopane']
training_data.loc[training_data[class_attr]==dict_map['Miombo open'],class_attr]=dict_map['Broadleaved (Semi-) deciduous closed forest']

training_data.to_file('Results/train_poly_848_20171124_class_merged.geojson', driver="GeoJSON")

### define query and feature layer function

In [ ]:
#set up our inputs to collect_training_data
zonal_stats = None
# Set up the inputs for the ODC query
time = ('2021')
# using spectral bands with 10~20 m spatial resolution
measurements = ['blue','green','red','red_edge_1','red_edge_2', 'red_edge_3','nir_1','nir_2','swir_1','swir_2']
resolution = (-10,10)
query = {
    'time': time,
    'measurements': measurements,
    'output_crs': output_crs,
    'resolution': resolution
}
# define a function to feature layers
def feature_layers(query): 
    # connect to the datacube so we can access DE Africa data
    dc = datacube.Datacube(app='feature_layers')
    
    # load Sentinel-2 analysis ready data
    ds = load_ard(dc=dc,
                  products=['s2_l2a'],
                  group_by='solar_day',
                  verbose=False,
                  **query)
    
    # calculate NDVI
    ds = calculate_indices(ds,
                           index=['NDVI'],
                           drop=False,
                           satellite_mission='s2')

    # calculate bi-monthly geomedian
    ds=ds.resample(time='2MS').map(xr_geomedian)
    
    # stack multi-temporal measurements and rename them
    n_time=ds.dims['time']
    list_measurements=list(ds.keys())
    list_stack_measures=[]
    for j in range(len(list_measurements)):
        for k in range(n_time):
            variable_name=list_measurements[j]+'_'+str(k)
            measure_single=ds[list_measurements[j]].isel(time=k).rename(variable_name)
            list_stack_measures.append(measure_single)
    ds_stacked=xr.merge(list_stack_measures,compat='override')
    return ds_stacked

### extract features

In [ ]:
# detect the number of CPUs
ncpus=round(get_cpu_quota())
print('ncpus = '+str(ncpus))

# collect training data
column_names, model_input = collect_training_data(
    gdf=training_data, # replace with gdf=training_data if you are extracting all the training data
    dc_query=query,
    ncpus=ncpus,
    field=class_attr,
    zonal_stats=zonal_stats,
    feature_func=feature_layers,
    return_coords=True)

### export training features

In [ ]:
# convert the data to geopandas dataframe
pd_training_features=pd.DataFrame(data=model_input,columns=column_names)
#set the name and location of the output file
output_file = "Results/Mozambique_training_features.txt"
#Export files to disk
pd_training_features.to_csv(output_file, header=True, index=None, sep=' ')